In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Library

# T.P.S Submission File

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


from sklearn.model_selection import StratifiedKFold 
# It means that you want to import and use layered K-Fold cross-validation.
#(계층화된 K-Fold 교차 검증을 import 해서 사용하겠다는 의미 )

from sklearn.preprocessing import StandardScaler , LabelEncoder 
# StandardScaler: Standardizes features by removing the mean and adjusting for unit variance.
# (StandardScaler : 평균을 제거하고 단위 분산에 맞게 조정하여 기능을 표준화.)
# LabelEncoder : Encodes the target label with a value between 0 and n_classes-1.
# (LabelEncoder : 0과 n_classes-1 사이의 값으로 대상 label을 인코딩.)

from sklearn.metrics import accuracy_score    
# accuracy_score: Indicates the accuracy classification score.
#(accuracy_score: 정확도 분류 점수를 의미한다.)

from sklearn.model_selection import train_test_split
from scipy.stats import mode # Meaning to return an array of modal (most common) values from the passed array.
                             #(전달된 배열에서 모달(가장 일반적인) 값의 배열을 반환한다는 의미.)


from lightgbm import LGBMClassifier


from matplotlib import ticker
import time
import warnings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:f}'.format)
warnings.filterwarnings('ignore')

### code source : https://www.kaggle.com/odins0n/tps-dec-eda-modeling

# Data Loading and Preperation (데이터 로드 및 준비)

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-dec-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-dec-2021/test.csv")
submission = pd.read_csv("../input/tabular-playground-series-dec-2021/sample_submission.csv")


train.drop(["Id"] , axis = 1 , inplace = True) 
# If 'drop' is used, the name of the specified column is deleted from 
# the row or column, and 'axis = 1' is used to designate the column. 
#(If '0', the row is designated.)
#('drop'을 사용하면 행이나 열에서 지정된 컬럼의 이름을 삭제, 'axis = 1'은 열을 지정하는 역할을 한다.('0'이면 행을 지정한다.))

test.drop(["Id"] , axis = 1 , inplace = True)
TARGET = 'Cover_Type'  
FEATURES = [col for col in train.columns if col not in ['id', TARGET]]
# 'List comprehension' is one of the ways to initialize a list, and it is used to initialize a list called 'FEATURES'.
# ('리스트 컴플리헨션'은 리스트를 초기화 하는 방법 중 하나인데 이걸 사용해서 'FEATURES'라는 리스트를 초기화 해줌.)

RANDOM_STATE = 12 

<a id="3.1"></a>
# Exploring Train Data (Train 데이터 탐색)

## Quick view of Train Data (Train 데이터 빨리 보기)

In [ ]:
train.head() # Use head() to output only the top 5 data in index order.
             # (head() 를 사용하여 인덱스 순으로 상위 5개의 데이터만 출력.)

In [ ]:
print(f'Number of rows in train data: {train.shape[0]}') 
# If f is used, 'Number of rows in test data:' is output as characters, 
# and if '.shape' is used, the total number of columns or rows is returned.
#(f를 사용하면 'Number of rows in test data:'를 문자로 출력하고, '.shape'를 사용하면 전체 열이나 행의 개수를 반환한다.)

print(f'Number of columns in train data: {train.shape[1]}')
print(f'No of missing values in train data: {sum(train.isna().sum())}')
# If 'isna().sum()' is used, the sum of 'missing values' is displayed.
# ('isna().sum()' 을 사용하면 '결측치'를 합한 값을 나타내준다.)

## Basic statistics of training data (훈련 데이터의 기본 통계)

In [ ]:
train.describe() # 'describe()' serves to output summary statistics
                 # ('describe()'는 요약 통계량을 출력하는 역할)

# Exploring Test Data (Test 데이터 탐색)

## Quick view of Test Data (Test 데이터의 빨리보기)

In [ ]:
test.head()

In [ ]:
print(f'Number of rows in test data: {test.shape[0]}')
print(f'Number of columns in test data: {test.shape[1]}')
print(f'No of missing values in test data: {sum(test.isna().sum())}')

<a id="3.3"></a>
# Submission File

In [ ]:
submission.head()

### Learn more about the train data and test data, and do more visualization and preprocessing.
(data source(데이터 출처) : https://www.kaggle.com/andrej0marinchenko/tps12-21-data-visualization)

In [ ]:
print("\n Name of the columns train data \n ", train.columns.tolist(), '\n')  # let's see the name of the columns train data
print('\n Name of the columns test data \n', test.columns.tolist(), '\n')  # let's see the name of the columns test data

In [ ]:
print('total number of columns train data', len(train.columns.tolist()))  # total number of columns (컬럼의 수)
print('total number of columns test data', len(test.columns.tolist()))  # total number of columns (컬럼의 수)

In [ ]:
train.head()  

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
df1 = train[0:50]
plt.figure(figsize=(30, 30))

sns.lineplot(data=df1) # Line graph visualization using seaborn (seaborn을 이용한 선 그래프 시각화)

In [ ]:
# train.value_counts()  # Counting the number of values (train 값의 개수)

In [ ]:
train.isnull().sum() #Check if there'is null values (널 값이 있는지 확인)

In [ ]:
train.iloc[:,:].hist(figsize=(18,18))  # Visualize the entire value of the train data as a histogram
plt.show()                               #(train 데이터의 전체 값을 히스토그램으로 시각화)

In [ ]:
cols = [e for e in test.columns if e not in ('Id')]
continous_features = cols[:10]
categorical_features = cols[10:]
# plot continous features  
i = 1
plt.figure()
fig, ax = plt.subplots(2, 5,figsize=(20, 12))
for feature in continous_features:
    plt.subplot(2, 5,i)
    sns.histplot(train[feature],color="blue", kde=True,bins=100, label='train_'+feature)
    sns.histplot(test[feature],color="olive", kde=True,bins=100, label='test_'+feature)
    plt.xlabel(feature, fontsize=9); plt.legend()
    i += 1
plt.show() 

In [ ]:
test.head()

In [ ]:
test.info()

In [ ]:
test.describe()

In [ ]:
test.iloc[:,:].hist(figsize=(18,18))
plt.show()

In [ ]:
df2 = test[0:50]
plt.figure(figsize=(30, 30))

sns.lineplot(data=df2)

In [ ]:
test.isnull().sum()

#### Target distibution (대상 분포)

In [ ]:
sns.catplot(x="Cover_Type", kind="count", palette="ch:.25", data=train)

In [ ]:
train.Cover_Type.value_counts()

In [ ]:
fig, ax = plt.subplots()
sns.countplot(x='Cover_Type', data=train, order=sorted(train['Cover_Type'].unique()), ax=ax)
ax.set_ylim(0, 2563000)
ax.set_title('Cover_Type Distribution', weight='bold')
plt.show()

<a id="4"></a>
# EDA

## Overview of Data (데이터 개요)

In [ ]:
train.iloc[:, :-1].describe().T.sort_values(by='std' , ascending = False)\
                     .style.background_gradient(cmap='turbo')\
                     .bar(subset=["max"], color='#3296B1')\
                     .bar(subset=["mean"], color='#1B71B1')\

## Continuos and Categorical Data Distribution (연속형 데이터와 명목형 데이터 분포)

In [ ]:
df = pd.concat([train[FEATURES], test[FEATURES]], axis=0)

cat_features = [col for col in FEATURES if df[col].nunique() < 9]
cont_features = [col for col in FEATURES if df[col].nunique() >= 9]

del df
print(f'Total number of features: {len(FEATURES)}')
print(f'Number of categorical features: {len(cat_features)}')
print(f'Number of continuos features: {len(cont_features)}')

plt.pie([len(cat_features), len(cont_features)],
       labels=['Categorical', 'Continuos'],
       colors=['#B14F3A', '#07E0C5'],
       textprops={'fontsize' : 15},
       autopct='%1.1f%%')

plt.show()

## Feature Distribution of Continous Features (연속형 데이터의 특징 분포)

In [ ]:
ncols = 5
nrows = int(len(cont_features) / ncols + (len(FEATURES) % ncols > 0))-1

fig, axes = plt.subplots(nrows, ncols, figsize=(18, 8), facecolor='#A171E0')

for r in range(nrows) :
    for c in range(ncols) :
        col = cont_features[r*ncols+c]
        sns.kdeplot(x=train[col], ax=axes[r, c], color='#6F8CB1', label='Train data')
        sns.kdeplot(x=test[col], ax=axes[r, c], color='#4A6BB1', label='Test data')
        axes[r, c].set_ylabel('')
        axes[r, c].set_xlabel(col, fontsize=10, fontweight='bold')
        axes[r, c].tick_params(labelsize=5, width=0.5)
        axes[r, c].xaxis.offsetText.set_fontsize(4)
        axes[r, c].yaxis.offsetText.set_fontsize(4)
        
plt.show()

## Feature Distribution of Categorical Features (명목형 데이터의 특징 분포)

In [ ]:
if len(cat_features) == 0 :
    print("No Categorical features")
else:
    ncols = 5
    nrows = int(len(cat_features) / ncols + (len(FEATURES) % ncols > 0))
    
    fig, axes = plt.subplots(nrows, ncols, figsize=(18, 45), facecolor='#DB92DB')
    
    for r in range(nrows):
        for c in range(ncols):
            if r*ncols+c >= len(cat_features):
                break
            col = cat_features[r*ncols+c]
            sns.countplot(x=train[col], ax=axes[r, c], color='#97A0DB', label='Train Data')
            sns.countplot(x=train[col], ax=axes[r, c], color='#DB48A1', label='Test Data')
            axes[r, c].set_ylabel('')
            axes[r, c].set_xlabel(col, fontsize=10, fontweight='bold')
            axes[r, c].tick_params(labelsize=5, width=0.6)
            axes[r, c].xaxis.offsetText.set_fontsize(4)
            axes[r, c].yaxis.offsetText.set_fontsize(4)
            
plt.show()

#### Soil_type7 and Soil_Type15 are all zero values (타입7과 타입15에는 값이 0이다.)

## Target Distribution (대상 분포)

In [ ]:
target_df = pd.DataFrame(train[TARGET].value_counts()).reset_index()
target_df.columns = [TARGET, 'count']
fig = px.bar(data_frame = target_df,
            x = 'Cover_Type',
            y = 'count',
            color = "count",
            color_continuous_scale="Emrld")

fig.show()
target_df.sort_values(by =TARGET , ignore_index = True)

#### There are total 7 different output classes (총 7개의 다른 클래스 출력이 있습니다.)

### Removing Unwanted Rows and columns (값이 없거나 필요없는 행,열 제거)

In [ ]:
train = train.drop(index = int(np.where(train["Cover_Type"] == 5 )[0]))    
train = train.drop(labels = ["Soil_Type7" , "Soil_Type8", "Soil_Type15"] ,axis = 1)
FEATURES.remove('Soil_Type7') 
FEATURES.remove('Soil_Type8')
FEATURES.remove('Soil_Type15')

In [ ]:
train.corr(method = "pearson").style.background_gradient(cmap='Blues')

In [ ]:
corr=train.corr() 
v=10
colmn = corr.nlargest(v, 'Cover_Type')['Cover_Type'].index 
xm = np.corrcoef(train[colmn].values.T)
sns.set(font_scale=1.25)
plt.figure(figsize=(18, 18))
hm = sns.heatmap(xm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10},yticklabels=colmn.values, xticklabels=colmn.values)
plt.show()

# Feature Engineering (데이터 전처리)

In [ ]:
train["mean"] = train[FEATURES].mean(axis=1)
train["std"] = train[FEATURES].std(axis=1)
train["min"] = train[FEATURES].min(axis=1)
train["max"] = train[FEATURES].max(axis=1)

test["mean"] = test[FEATURES].mean(axis=1)
test["std"] = test[FEATURES].std(axis=1)
test["min"] = test[FEATURES].min(axis=1)
test["max"] = test[FEATURES].max(axis=1)

FEATURES.extend(['mean', 'std', 'min', 'max'])

# Modeling (모델링)

In [ ]:
scaler = StandardScaler()
for col in FEATURES:
    train[col] = scaler.fit_transform(train[col].to_numpy().reshape(-1,1))
    test[col] = scaler.transform(test[col].to_numpy().reshape(-1,1))
    
X = train[FEATURES].to_numpy().astype(np.float32)
y = train[TARGET].to_numpy().astype(np.float32)
X_test = test[FEATURES].to_numpy().astype(np.float32)

# LGBM Classifier (Light BGM 분류기)

In [ ]:
lgb_params = {
    'objective' : 'multiclass',
    'metric' : 'multi_logloss',
    'device' : 'gpu',
}


lgb_predictions = []
lgb_scores = []

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
for fold, (train_idx, valid_idx) in enumerate(kf.split(X = X, y = y)):

    print(10*"=", f"Fold={fold+1}", 10*"=")
    start_time = time.time()
    x_train = X[train_idx, :]
    x_valid = X[valid_idx, :]
    y_train = y[train_idx]
    y_valid = y[valid_idx]
    
    model = LGBMClassifier(**lgb_params)
    model.fit(x_train, y_train,
          early_stopping_rounds=200,
          eval_set=[(x_valid, y_valid)],
          verbose=0)
    
    preds_valid = model.predict(x_valid)
    acc = accuracy_score(y_valid,  preds_valid)
    lgb_scores.append(acc)
    run_time = time.time() - start_time
    print(f"Fold={fold+1}, acc: {acc:.8f}, Run Time: {run_time:.2f}")
    test_preds = model.predict(X_test)
    lgb_predictions.append(test_preds)
    
print("Mean Accuracy :", np.mean(lgb_scores))

# Submission (제출)

### LGBM Classifier Submission (Light BGM 분류기 제출)

In [ ]:
lgb_submission = submission.copy()
lgb_submission['Cover_Type'] = np.squeeze(mode(np.column_stack(lgb_predictions),axis = 1)[0]).astype('int')
lgb_submission.to_csv("lgb-subs.csv",index=None)
lgb_submission.head()